# Chuẩn bị thư viện

In [1]:
!pip3 install selenium
!mkdir -p data
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import os
import json


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [3]:
MAGAZINE_NAME = "vietnamnet"
HOME_PAGE = "https://vietnamnet.vn/"

# Thu thập dữ liệu

In [4]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [5]:
# Lấy hết tất cả thể loại từ đây
cats = []

#### CODE
# row_menu = driver.find_elements(by=By.CLASS_NAME, value="header_submenu-content")
sub_menu = driver.find_element(by=By.CLASS_NAME, value="header_submenu-content-list")

cat_menus = sub_menu.find_elements(by=By.TAG_NAME, value="a")

for cat_name in cat_menus:
    cat = cat_name.get_attribute("title").strip()
    href = cat_name.get_attribute("href").strip()
    
    # bỏ qua logo-premium, logo-htvn
    if cat == "premium vietnamnet" or cat == "Hành trình Việt Nam":
        continue
    else:
        cats.append({"cat_names": cat, "url": href})
####

In [6]:
cats, len(cats)

([{'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'},
  {'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_names': 'Dân tộc và Tôn giáo',
   'url': 'https://vietnamnet.vn/dan-toc-ton-giao'},
  {'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_names': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_names': 'Văn hóa - Giải trí',
   'url': 'https://vietnamnet.vn/van-hoa-giai-tri'},
  {'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_names': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'},
  {'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_names': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_names': 'Du lịch', 'url': 

In [7]:
TARGET_CATS = [
    "Chính trị",
    "Thời sự",
    "Kinh doanh",
    "Văn hóa - Giải trí",
    "Sức khỏe",
    "Pháp luật",
    "Thế giới",
    "Công nghệ",
    "Thể thao",
    "Du lịch",
    "Giáo dục"
]

filtered_cats = [c for c in cats if c['cat_names'] in TARGET_CATS]

print(len(filtered_cats))
for c in filtered_cats:
    print(c)


11
{'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'}
{'cat_names': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'}
{'cat_names': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'}
{'cat_names': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'}
{'cat_names': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'}
{'cat_names': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'}
{'cat_names': 'Văn hóa - Giải trí', 'url': 'https://vietnamnet.vn/van-hoa-giai-tri'}
{'cat_names': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'}
{'cat_names': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'}
{'cat_names': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'}
{'cat_names': 'Du lịch', 'url': 'https://vietnamnet.vn/du-lich'}


# Thu thập URL bài báo

In [8]:
NUM_ARTICLES_PER_CAT = 10 # có thể tăng lên

DATA_URL_FILE = "data/vietnamnet_url.json"

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"
driver = webdriver.Chrome(options=chrome_options)

## Thử nghiệm

In [32]:
driver = webdriver.Chrome(options=chrome_options)

In [33]:
filtered_cats[1]['url']

'https://vietnamnet.vn/thoi-su'

In [34]:
driver.get(filtered_cats[1]['url'])

In [35]:
title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
print(title_news[0].text)

Màn đối đáp 'cực gắt' khiến kẻ giả danh cán bộ thuế nghẹn ngào thú tội


In [36]:
url_new = title_news[0].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
print(url_new)

https://vietnamnet.vn/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-2433305.html


In [38]:
# Tìm layout thứ 2 của trang web

#### CODE
page_numbers = driver.find_element(by=By.CLASS_NAME, value="topStory-2nd")
next_page = page_numbers.find_elements(by=By.TAG_NAME, value="a")
# print(len(next_page))
# print(next_page[0].get_attribute("href"))
# print(next_page[1].get_attribute("href"))
# print(next_page[2].get_attribute("href"))
# print(next_page[3].get_attribute("href"))
# print(next_page[4].get_attribute("href"))
# print(next_page[5].get_attribute("href"))
next_page = next_page[3].get_attribute("href")
print(next_page)

https://vietnamnet.vn/du-bao-thoi-tiet-20-8-2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-2433768.html


In [39]:
driver.close()

## Chạy thật

In [19]:
driver = webdriver.Chrome(options=chrome_options)

In [20]:
NUM_ARTICLES_PER_CAT = 3
crawled_urls = set()

def get_avatar_url_by_article_url(driver, article_url):
    try:
        avatar_divs = driver.find_elements(by=By.CLASS_NAME, value="verticalPost__avt")
        
        for avatar_div in avatar_divs:
            try:
                link_element = avatar_div.find_element(by=By.TAG_NAME, value="a")
                href = link_element.get_attribute("href")
                
                # Nếu href match với article_url (có thể là relative hoặc absolute)
                if href and (href == article_url or article_url.endswith(href) or href in article_url):
                    img_element = avatar_div.find_element(by=By.TAG_NAME, value="img")
                    print(f"Found avatar for {article_url}: {img_element.get_attribute('src')}")
                    return img_element.get_attribute("src")
                
            except NoSuchElementException:
                continue
        
        return None
    except Exception as e:
        print(f"Error getting avatar for {article_url}: {e}")
        return None

def crawl_each_category_url(driver, category_url):
    all_data = []
    url = category_url
    
    while len(all_data) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CLASS_NAME, value="vnn-title")
        
        for title in title_news:
            try:
                url_new = title.find_element(by=By.TAG_NAME, value="a").get_attribute("href")
                if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:
                    avatar_url = get_avatar_url_by_article_url(driver, url_new)
                    
                    article_data = {
                        "url": url_new,
                        "url_img": avatar_url
                    }
                    
                    all_data.append(article_data)
                    crawled_urls.add(url_new)

                    if len(all_data) >= NUM_ARTICLES_PER_CAT:
                        break
                        
            except (StaleElementReferenceException, NoSuchElementException):
                continue

            except Exception as e:
                print(f"Error processing article: {e}")
        
        if len(all_data) >= NUM_ARTICLES_PER_CAT:
            break
        
        try:
            next_pages = driver.find_element(by=By.CLASS_NAME, value="pagination__list")
            url = next_pages.find_elements(by=By.TAG_NAME, value="a")[6].get_attribute("href")
        except (NoSuchElementException, IndexError):
            print("Cannot find next page, stopping crawl for this category")
            break
    
    return all_data

    

In [21]:
saved_cats = {}
for cat in cats:
    cat_name = cat['cat_names']
    url = cat['url']
    if cat_name in TARGET_CATS:
        print(f"You are at {cat}.")
        article_data = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = article_data


with open(DATA_URL_FILE, "w", encoding="utf-8") as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_names': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'}.
Found avatar for https://vietnamnet.vn/bo-quoc-phong-de-xuat-thanh-lap-ban-chi-dao-phong-khong-nhan-dan-trung-uong-2433796.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/bo-quoc-phong-de-xuat-thanh-lap-ban-chi-dao-phong-khong-nhan-dan-trung-uong-4199.jpg?width=550&s=lTCQ8imV0ar-QpJvYBrflA
Found avatar for https://vietnamnet.vn/phu-nhan-chu-tich-nuoc-va-hoang-hau-bhutan-van-canh-chua-tran-quoc-duoi-mua-2433621.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/phu-nhan-chu-tich-nuoc-va-hoang-hau-bhutan-van-canh-chua-tran-quoc-duoi-mua-dau-thu-2104.jpg?width=360&s=UyWw2dLkheZzTlxi1UnNTg
Found avatar for https://vietnamnet.vn/xa-mien-nui-thanh-hoa-kho-khan-ve-nhan-su-can-bo-lam-viec-den-tan-khuya-2431628.html: https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/xa-mien-nui-thanh-hoa-thieu-nhan-su-can-bo-lam-viec-den-tan-

In [21]:
len(crawled_urls)

33

# Thu thập thông tin bài báo

In [40]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vietnamnet_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data\\vietnamnet"

!mkdir -p $DATA_FOLDER_OUTPUT

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [41]:
# Đọc file url

with open(FILE_URL_PATH, "r", encoding="utf-8") as fIn:
    url_data = json.load(fIn)

len(url_data)

11

## Chạy thử nghiệm

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
        {
            "url": "https://vietnamnet.vn/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-2433305.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/19/man-doi-dap-cuc-gat-khien-ke-gia-danh-can-bo-thue-nghen-ngao-thu-toi-1546.gif?width=550&s=o2b7BQcMjEroZaFOnitZTw"
        },
        {
            "url": "https://vietnamnet.vn/nguy-co-rinh-rap-tu-diem-den-loi-di-tu-mo-qua-duong-sat-o-ninh-binh-2432916.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/17/hanh-dong-lieu-linh-cua-nguoi-dan-ninh-binh-khi-song-gan-duong-ray-tau-hoa-998.jpg?width=360&s=Xo-IcBTRm58t6H6s-S8iPg"
        },
        {
            "url": "https://vietnamnet.vn/du-bao-thoi-tiet-20-8-2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-2433768.html",
            "url_img": "https://static-images.vnncdn.net/vps_images_publish/000001/000003/2025/8/20/2025-mien-bac-nang-den-34-do-van-xuat-hien-mua-to-cuc-bo-255.jpg?width=360&s=R4M4D3fBLPTkOTsLldQ3mA"
        }
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[2]

In [43]:
driver.get(SAMPLE_ARTICLE_URL)

In [44]:
# Title
driver.find_element(by=By.CSS_SELECTOR, value="h1.content-detail-title").text

'Tử hình kẻ tấn công khiến cảnh sát cơ động hi sinh, người mẹ bị hại khóc, nói lời xót xa'

In [45]:
# Tìm kiếm mô tả 
driver.find_element(by=By.CLASS_NAME, value="sm-sapo-mb-0").text

'Sau hơn 5 tháng gây ra vụ án khiến chiến sĩ Cảnh sát cơ động hi sinh trong khi làm nhiệm vụ ở TP Vũng Tàu (cũ), Dương Hữu Trí bị đưa ra xét xử với 4 tội danh nghiêm trọng và bị tuyên phạt mức án cao nhất là tử hình.'

In [46]:
# Thu thập thể loại
lis_cat = []
sub_cat = driver.find_element(by=By.CLASS_NAME, value="sm-show-time")
lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
for cat in lis_cats:
    category = cat.get_attribute("title")
    if len(category):
        lis_cat.append(category)
lis_cat

['Pháp luật',
 'Chính trị',
 'Thời sự',
 'Kinh doanh',
 'Dân tộc và Tôn giáo',
 'Thể thao',
 'Giáo dục',
 'Thế giới',
 'Đời sống',
 'Văn hóa - Giải trí',
 'Sức khỏe',
 'Công nghệ',
 'Pháp luật',
 'Xe',
 'Bất động sản',
 'Du lịch',
 'Bạn đọc',
 'Ký sự pháp đình']

In [47]:
#Thu thập ngày
#### CODE HERE
publisd_date = driver.find_element(by=By.CLASS_NAME, value="bread-crumb-detail__time").text
publisd_date
####


'thứ hai, 18/08/2025 - 18:14'

In [48]:
# Tìm kiếm contents
#### CODE HERE
article = driver.find_element(by=By.CLASS_NAME, value="main-content")
children = article.find_elements(by=By.XPATH, value="./*")
####

In [49]:
# Thu thập contents
contents = []
author = "Unknown"
is_slide_show = False

for idx, child in enumerate(children):
    text = child.text.strip()
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

        #### CODE HERE
        contents.append(text)
        ####

    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        #### CODE HERE
        if len(text) > 0:
            if len(text) <= 100:
                contents.append(f"[{text}]")
            else:
                contents.append(text)
        ####

    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [50]:
contents

['Chiều 18/8, TAND TPHCM cơ sở 1 (tỉnh Bà Rịa - Vũng Tàu cũ) mở phiên tòa xét xử sơ thẩm bị cáo Dương Hữu Trí (38 tuổi, trú phường 7, TP Vũng Tàu cũ - nay là phường Tam Thắng, TPHCM) về các tội "Giết người", "Cố ý gây thương tích", "Chống người thi hành công vụ" và "Hủy hoại tài sản".',
 '[Bị cáo Dương Hữu Trí tại phiên tòa. Ảnh: Quang Hưng]',
 'Trí là đối tượng từng dùng dao tấn công khiến hạ sĩ Nguyễn Ngọc Minh Nhật (SN 2001, quê Phú Yên cũ) chiến sĩ Cảnh sát cơ động thuộc Phòng CSCĐ tỉnh Bà Rịa - Vũng Tàu (cũ) - hi sinh hồi tháng 3 vừa qua, gây xôn xao dư luận.',
 'Theo cáo trạng, Dương Hữu Trí là đối tượng được quản lý sau cai nghiện ma túy tại nơi cư trú do UBND phường 7, TP Vũng Tàu (cũ) quản lý từ ngày 19/6/2023 đến 19/6/2025.',
 'Ngày 17/2/2025, theo kế hoạch, Công an phường 7 đến nhà mời Trí xét nghiệm ma túy nhưng đối tượng chống đối, không chấp hành. Sau đó, công an địa phương đưa vào diện theo dõi, xử lý.',
 'Khoảng 7h ngày 3/3, bốn cán bộ Công an phường 7 tiếp tục đến nhà 

In [51]:
# Thu thập tác giả
is_author_maybe_last = False
authors = []
try:
    #### CODE HERE
    name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
    author_names = name.find_elements(by=By.CLASS_NAME, value="name")
    for author_name in author_names:
        author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
        authors.append(author)
    ####
except NoSuchElementException:
    is_author_maybe_last = True

# Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
if is_author_maybe_last:
    if len(contents):
        last_content = contents[-1]
        if last_content.find('-') != -1 and last_content.find('-') <= 30:
            authors = last_content[:(last_content.find('-')-1)]
        else:
            authors = "Unknown"

In [52]:
authors

['Quang Hưng']

In [53]:
driver.close()

## Chạy thật

In [ ]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    title = driver.find_element(by=By.CSS_SELECTOR, value="h1.content-detail-title").text # Title
    description = driver.find_element(by=By.CLASS_NAME, value="sm-sapo-mb-0").text # Description
    ####

    # Thu thập thể loại
    #### CODE HERE
    lis_cat = []
    sub_cat = driver.find_element(by=By.CLASS_NAME, value="sm-show-time")
    lis_cats = sub_cat.find_element(by=By.CSS_SELECTOR, value="ul").find_elements(by=By.TAG_NAME, value="a")
    for cat in lis_cats:
        category = cat.get_attribute("title")
        if len(category):
            lis_cat.append(category)
    
    main_cat = lis_cat[0]
    if len(lis_cat) > 1:
        sub_cat = lis_cat[1]
    else:
        sub_cat = "None"
    ####


    # Thu thập published date
    published_date = driver.find_element(by=By.CLASS_NAME, value="bread-crumb-detail__time").text

    # Thu thập content bài báo
    #### CODE HERE
    # Locate phần viết content
    article = driver.find_element(by=By.CLASS_NAME, value="main-content")
    # Lấy hết các đầu mục con của bài báo
    children = article.find_elements(by=By.XPATH, value="./*")
    
    # Thu thập contents
    contents = []
    author = "Unknown"
    is_slide_show = False

    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph
            contents.append(text)
            ####

        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            #### CODE HERE
            if len(text) > 0:
                if len(text) <= 100:
                    contents.append(f"[{text}]")
                else:
                    contents.append(text)
            ####

        elif child.tag_name == "table": # Do nothing rightnow
            pass
    ####

    # Thu thập tác giả
    #### CODE HERE
    is_author_maybe_last = False
    authors = []
    try:
        #### CODE HERE
        name = driver.find_element(by=By.CLASS_NAME, value="article-detail-author-wrapper")
        author_names = name.find_elements(by=By.CLASS_NAME, value="name")
        for author_name in author_names:
            author = author_name.find_element(by=By.CSS_SELECTOR, value="a").get_attribute("title")
            authors.append(author)
        ####
    except NoSuchElementException:
        is_author_maybe_last = True

    # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
    if is_author_maybe_last:
        if len(contents):
            last_content = contents[-1]
            if last_content.find('-') != -1 and last_content.find('-') <= 30:
                authors = last_content[:(last_content.find('-')-1)]
            else:
                authors = "Unknown"


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": published_date,
            "author": author
        }
    }
